# Tests for Activity 7
## Adding Keys and Constraints to the Yote DB

In [3]:
# Testing Part 1 (Setup)
import sqlite3
from nose.tools import assert_equal

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())

# Running all the code in act7insert.sql
with open('act7insert.sql') as file:
    cur.executescript(file.read())

# Checking that the tables are created 
tables = {t[0] for t in cur.execute("SELECT tbl_name FROM sqlite_master;").fetchall()}

assert_equal(tables, {'CourseDetails', 'CourseMeetings', 'CourseStudent', 'Courses', 'Yotes'})

# Checking the column names
con.close()

In [4]:
# Testing Part 2
import sqlite3
from nose.tools import assert_equal, assert_in

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())

# Checking Keys and Constraints:
table_info = cur.execute("PRAGMA table_info('Yotes');").fetchall()

primary_keys = {t[1] for t in table_info if t[-1]}
assert_equal(primary_keys, {'YoteId'}, 'Checking the Primary Key')

not_nulls = {t[1] for t in table_info if t[3]}
assert_in('Email', not_nulls, 'Checking the attributes that cannot be null')

# Checking Emails are unique:
try:
    cur.executescript("""  
    INSERT INTO Yotes (Email) VALUES ('a@b.edu'), ('a@b.edu');
    """)
except Exception as err:
    message2 = str(err)

assert_equal(message2, 'UNIQUE constraint failed: Yotes.Email')

con.close()

In [5]:
# Testing Part 3a
import sqlite3
from nose.tools import assert_equal, assert_greater

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())

#Checking Keys:
table_info = cur.execute("PRAGMA table_info('CourseDetails');").fetchall()

primary_keys = {t[1] for t in table_info if t[-1]}
assert_equal(primary_keys, {'CourseSubject', 'CourseNumber'}, 'Checking the Primary Key')

mystery = {t[1] for t in table_info if t[3]}
assert_greater(len(mystery), 1, 'Checking if you added the extra constraint(s) to enforce the Primary Key constraint')

con.close()

In [6]:
# Testing Part 3b
import sqlite3
from nose.tools import assert_equal

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())

#Checking Keys:
table_info = cur.execute("PRAGMA table_info('CourseStudent');").fetchall()

primary_keys = {t[1] for t in table_info if t[-1]}
assert_equal(primary_keys, {'CourseId', 'StudentId'}, 'Checking the Primary Key')

con.close()

In [7]:
# Testing Part 3c
import sqlite3
from nose.tools import assert_equal, assert_not_equal

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())

#Checking Keys:
tables = {t[0] for t in cur.execute("SELECT tbl_name FROM sqlite_master;").fetchall()}

for table in tables:
    table_info = cur.execute(f"PRAGMA table_info({table});").fetchall()
    primary_keys = {t[1] for t in table_info if t[-1]}
    assert_not_equal(len(primary_keys), 0, 'Checking that each table has a primary key')

# There are hidden tests which will check if you have the correct Primary Keys.

con.close()

In [8]:
# Testing Part 3d part 1
import sqlite3
from nose.tools import assert_in

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())

#Checking Keys:
fk_info = {t for t in cur.execute("PRAGMA foreign_key_list('Courses');").fetchall()}

foreign_keys = {(fk[2], fk[3], fk[4]) for fk in fk_info}
assert_in(('Yotes', 'InstructorId', 'YoteId'), foreign_keys, 'Checking a FK')

con.close()

In [9]:
# Testing Part 3d part 2
import sqlite3
from nose.tools import assert_in, assert_equal, assert_not_equal

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())

#Checking Keys:
fk_info = {t for t in cur.execute("PRAGMA foreign_key_list('Courses');").fetchall()}

foreign_keys = {(fk[2], fk[3], fk[4]) for fk in fk_info}
assert_in(('CourseDetails', 'CourseSubject', 'CourseSubject',), foreign_keys, 'Checking CourseSubject FK')
assert_in(('CourseDetails', 'CourseNumber', 'CourseNumber',),   foreign_keys, 'Checking CourseNumber FK')

fks = [fk for fk in fk_info if fk[2] == 'CourseDetails']
assert_equal(len(fks), 2,              'Checking the reference to CourseDetails is setup as a *compound* FK')
assert_equal(fks[0][0], fks[1][0],     'Checking the reference to CourseDetails is setup as a *compound* FK')
assert_not_equal(fks[0][1], fks[1][1], 'Checking the reference to CourseDetails is setup as a *compound* FK')

con.close()

In [10]:
# Testing Part 3e
import sqlite3
from nose.tools import assert_equal

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())

#Checking Keys:
tables = {t[0] for t in cur.execute("SELECT tbl_name FROM sqlite_master;").fetchall()}
fk_count = sum([len(cur.execute(f"PRAGMA foreign_key_list({table});").fetchall()) for table in tables])

assert_equal(fk_count, 6, "Checking the number of FK's")

# There are hidden tests which will check if you have the correct Foreign Keys.

con.close()

In [11]:
# Testing Part 3f
import sqlite3
from nose.tools import assert_equal, assert_not_equal

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())

#Checking Actions:
tables = {t[0] for t in cur.execute("SELECT tbl_name FROM sqlite_master;").fetchall()}
fk_info = {f for table in tables for f in cur.execute(f"PRAGMA foreign_key_list({table});").fetchall()}

actions = {fk[-2] for fk in fk_info if fk[2] == 'CourseDetails'}
for a in actions:
    assert_not_equal(a, 'NO ACTION', 'Checking ON DELETE actions for references to CourseDetails')

action = [fk[-2] for fk in fk_info if fk[3] == 'InstructorId'][0]
assert_not_equal(action, 'NO ACTION', 'Checking ON DELETE action for InstructorId')

# There are hidden tests which will check if you have the correct actions.

con.close()

In [12]:
# Testing Part 3g
import sqlite3
from nose.tools import assert_in

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())

# Checking constraint:
table_info = cur.execute("PRAGMA table_info('CourseDetails');").fetchall()

not_nulls = {t[1] for t in table_info if t[3]}
assert_in('Title', not_nulls, 'Checking the attributes that cannot be null')

con.close()

In [13]:
# Testing Part 3h
import sqlite3
from nose.tools import assert_in

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())

# Checking constraint:
tests = [(11, 'CHECK constraint failed'),
         (-1, 'CHECK constraint failed'),
         (10, 'No Error'),
         (0, 'No Error'),
         (4, 'No Error')]
num = 100

for credit, msg in tests:
    num += 1
    try:
        cur.executescript(f"""  
        INSERT INTO CourseDetails (CourseSubject, CourseNumber, Title, Credits) VALUES ('ABC', '{num}', 'Computers', {credit});
        """)
        message = 'No Error'
    except Exception as err:
        message = str(err)

    assert_in(msg, message, f'Checking credits = {credit}')

con.close()

In [14]:
# Testing Part 3i
import sqlite3
from nose.tools import assert_equal

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())

# Checking default:
table_info = cur.execute("PRAGMA table_info('Courses');").fetchall()

for t in table_info:
    if t[1] == 'Section':
        default = t[-2]

assert_equal(default, '1', 'Checking the default')

con.close()

In [15]:
# Testing Part 3j
import sqlite3
from nose.tools import assert_in

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())

# Checking constraint:
try:
    cur.executescript(f"""  
    INSERT INTO CourseMeetings (StartTime, EndTime) VALUES ('10:00', '09:50');
    """)
    message = 'No Error'
except Exception as err:
    message = str(err)

assert_in('CHECK constraint failed', message)

con.close()

In [16]:
# Testing Part 3k
import sqlite3
from nose.tools import assert_in

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())

# Checking constraint:
try:
    cur.executescript(f"""  
    INSERT INTO Courses (CourseSubject, CourseNumber, Term) VALUES ('ABC', '123', 'Sp');
    """)
    message = 'No Error'
except Exception as err:
    message = str(err)

assert_in('CHECK constraint failed', message, "Checking Term = 'Sp'")

try:
    cur.executescript(f"""  
    INSERT INTO Courses (CourseSubject, CourseNumber, Term) VALUES ('ABC', '123', 'F');
    """)
    message = 'No Error'
except Exception as err:
    message = str(err)

assert_in('CHECK constraint failed', message, "Checking Term = 'F'")

try:
    cur.executescript(f"""  
    INSERT INTO Courses (CourseSubject, CourseNumber, Term) VALUES ('ABC', '123', 'Fall');
    """)
    message = 'No Error'
except Exception as err:
    message = str(err)

assert_in('No Error', message, "Checking Term = 'Fall'")

con.close()

In [17]:
# Testing Part 4
import sqlite3
from nose.tools import assert_equal, assert_greater

# Setting up sqlite3 for unit tests
con = sqlite3.connect(':memory:')
cur = con.cursor()

# Running all the code in act7create.sql
with open('act7create.sql') as file:
    cur.executescript(file.read())
    
# Running all the code in act7insert.sql
with open('act7insert.sql') as file:
    cur.executescript(file.read())
    
# Checking there is data in the database
tables = {t[0] for t in cur.execute("SELECT tbl_name FROM sqlite_master;").fetchall()}
assert_equal(tables, {'CourseDetails', 'CourseMeetings', 'CourseStudent', 'Courses', 'Yotes'})

for table in tables:
    rows = cur.execute(f"SELECT * FROM {table};").fetchall()
    assert_greater(len(rows), 2)

# Checking there are no FK violators
fk_violators = cur.execute("PRAGMA foreign_key_check;").fetchall()
assert_equal(len(fk_violators), 0, "Checking that there are no FK violations")

con.close()